#### **Trabalho de Processamento de Big Data | Licenciatura em Ciência de Dados 2023/24 | CDB1**

Docente: João Oliveira<br><br>

- David Franco, nº110733

- Felipe Pereira, nº110861

- João Dias, nº110305

- Samuel Ricardo, nº110884<br><br>

https://www.kaggle.com/datasets/PROPPG-PPG/hourly-weather-surface-brazil-southeast-region

*Version 9 (10.11 GB) | Created by John Holz | Data Update 2023/01/30*

### Aplicação dos modelos gerados no passo anterior

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml import PipelineModel
import pyspark.sql.functions as F
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator
from pyspark.ml.tuning import TrainValidationSplitModel

In [2]:
# Criar a sessão de Spark

spark = SparkSession.builder \
    .appName("WeatherBrazilModelingApplication") \
    .config("spark.sql.shuffle.partitions", 512) \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

In [3]:
! ls

all_stations.parquet
classification-validation-set.parquet
datasets_originais
model-RandomForestRegression
model-svm
n1_trabalho_final_limpeza.ipynb
n2_trabalho_final_analise_exploratoria.ipynb
n3_trabalho_final_modelos.ipynb
n4_trabalho_final_aplica_modelo.ipynb
n5_trabalho_final_relatorio.ipynb
pbd_projeto_2024.pdf
pipeline-RandomForestRegression
pipeline-svm
regression-validation-set.parquet


### Previsão da temperatura

In [4]:
df_validation = spark.read.parquet("regression-validation-set.parquet")

Leitura dos dados de validação do arquivo Parquet.

In [5]:
pipeline_model = PipelineModel.load("model-RandomForestRegression")

Carrega o pipeline treinado do disco.

In [6]:
# fazer as previsões
df_prediction = pipeline_model.transform(df_validation)

# verificar o schema
df_prediction.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Hour: string (nullable = true)
 |-- TotalHourlyPrecipitationMm: double (nullable = true)
 |-- HourlyStationLevelAtmosphericPressureMb: double (nullable = true)
 |-- LastHourMaxAtmosphericPressureMb: double (nullable = true)
 |-- LastHourMinAtmosphericPressureMb: double (nullable = true)
 |-- HourlyDryBulbAirTemperatureC: double (nullable = true)
 |-- DewPointTemperatureC: double (nullable = true)
 |-- LastHourMaxDewPointTemperatureC: double (nullable = true)
 |-- LastHourMinDewPointTemperatureC: double (nullable = true)
 |-- LastHourMaxRelativeHumidityPercentage: integer (nullable = true)
 |-- LastHourMinRelativeHumidityPercentage: integer (nullable = true)
 |-- HourlyRelativeHumidityPercentage: integer (nullable = true)
 |-- HourlyWindDirectionRadiusDegrees: integer (nullable = true)
 |-- MaximumWindGustMs: double (nullable = true)
 |-- HourlyWindSpeedMs: double (nullable = true)
 |-- Region: string (nullable = true)
 |-- State: string (null

Faz previsões utilizando o modelo carregado anteriormente e aplica essas previsões ao conjunto de dados de validação. Depois, faz print do schema do dataframe resultante para verificar a estrutura dos dados previstos.

In [7]:
# verificar as previsões vs resultados reais
df_prediction.select('features', 'prediction', 'AverageTemperatureC').show(truncate=False)

+-----------------------------------------------------------------------------------------------+------------------+-------------------+
|features                                                                                       |prediction        |AverageTemperatureC|
+-----------------------------------------------------------------------------------------------+------------------+-------------------+
|[0.0,26.784536226022087,4.389901860146138,3.8168979116104427,0.5299651423741525]               |25.047309782806053|25.35              |
|[0.47455989094119655,25.582837192950308,3.885798297258544,4.461309247336881,1.1006968341617014]|21.756493704904262|20.85              |
|[0.0,23.312961241592504,3.1926558982881006,3.8168979116104427,1.1516550209284466]              |19.300048413997672|19.1               |
|[0.0,24.72829565832149,4.179858708942974,4.510879350085069,2.017944195963119]                  |20.825945326988112|21.95              |
|[0.0,26.864649494893538,4.72597090207120

Seleciona as colunas `features`, `prediction` e `AverageTemperatureC` do dataframe de previsões e exibe os valores, em prol de comparar as previsões do modelo com os resultados reais da temperatura média, obtendo valores muito semelhantes.

In [8]:
# Previsões no conjunto de validação
df_validation_prediction = pipeline_model.transform(df_validation)

# Calcular métricas no conjunto de validação
print("\nMétricas no conjunto de validação:")
for metric in ["rmse", "mse", "mae", "r2"]:
    evaluator = RegressionEvaluator(labelCol="AverageTemperatureC", 
                                    predictionCol="prediction", 
                                    metricName=metric)
    value = evaluator.evaluate(df_validation_prediction)
    print(f"Validation Metric {metric} = {value}")


Métricas no conjunto de validação:
Validation Metric rmse = 2.348234359432686
Validation Metric mse = 5.514204606820237
Validation Metric mae = 1.6548994294137063
Validation Metric r2 = 0.8193569327069066


Comparando os resultados obtidos nos dois conjuntos de métricas (com o notebook anterior), observamos que as métricas de validação e teste apresentam valores muito próximos entre si. Isso sugere que o modelo tem uma capacidade consistente de generalização, ou seja, ele é capaz de fazer previsões precisas tanto em dados que ele já viu como em dados novos (conjunto de validação).

Concluímos que o modelo de regressão foi bem ajustado aos dados de temperatura, com valores de RMSE, MSE e MAE em torno de 2°C e um R² de aproximadamente 0.82, indicando que cerca de 82% da variabilidade nos dados é explicada pelo modelo. Isso sugere que o modelo é robusto e pode ser aplicado com confiança para prever a temperatura média.

In [9]:
# guardar os resultados
df_prediction.write.mode("overwrite").parquet("regression_model_results.parquet")

Os resultados das previsões feitas pelo modelo de regressão foram gravados no formato Parquet no arquivo 'regression_model_results.parquet'. O modo 'overwrite' foi utilizado para substituir qualquer arquivo existente com o mesmo nome.

### Previsão da precipitação

In [10]:
# Ler os dados de validação do arquivo Parquet
df_validation = spark.read.parquet("classification-validation-set.parquet")

Leitura dos dados de validação do arquivo Parquet.

In [11]:
# Carregar o pipeline treinado do disco
pipeline_model = TrainValidationSplitModel.load("model-svm")

Carrega o pipeline treinado do disco.

In [12]:
# fazer as previsões
df_prediction = pipeline_model.transform(df_validation)

# verificar o schema
df_prediction.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Hour: string (nullable = true)
 |-- HourlyStationLevelAtmosphericPressureMb: double (nullable = true)
 |-- LastHourMaxAtmosphericPressureMb: double (nullable = true)
 |-- LastHourMinAtmosphericPressureMb: double (nullable = true)
 |-- HourlyDryBulbAirTemperatureC: double (nullable = true)
 |-- DewPointTemperatureC: double (nullable = true)
 |-- LastHourMaxDewPointTemperatureC: double (nullable = true)
 |-- LastHourMinDewPointTemperatureC: double (nullable = true)
 |-- LastHourMaxRelativeHumidityPercentage: integer (nullable = true)
 |-- LastHourMinRelativeHumidityPercentage: integer (nullable = true)
 |-- HourlyRelativeHumidityPercentage: integer (nullable = true)
 |-- HourlyWindDirectionRadiusDegrees: integer (nullable = true)
 |-- MaximumWindGustMs: double (nullable = true)
 |-- HourlyWindSpeedMs: double (nullable = true)
 |-- Region: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Station: string (nullable = true)
 |-- S

Faz previsões utilizando o modelo de pipeline treinado anteriormente no conjunto de validação e faz print da estrutura do dataframe.

In [13]:
df_prediction.select('features', 'prediction', 'Rains').show(truncate=False)

+------------------------------------------+----------+-----+
|features                                  |prediction|Rains|
+------------------------------------------+----------+-----+
|[964.1,21.5,89.0,56.0,24.25]              |1.0       |1    |
|[919.1,20.0,89.0,338.0,22.2]              |1.0       |1    |
|[968.8,20.7,93.0,201.0,22.7]              |1.0       |1    |
|[978.8,22.5,95.0,246.0,23.4]              |1.0       |1    |
|[963.2,22.8,96.0,99.0,23.85]              |1.0       |1    |
|[1014.1,19.9,93.0,223.0,21.35]            |1.0       |1    |
|[963.2,20.6,85.0,65.0,23.299999999999997] |1.0       |1    |
|[968.6,21.3,96.0,320.0,22.0]              |1.0       |1    |
|[966.7,20.5,95.0,323.0,21.299999999999997]|1.0       |1    |
|[954.3,21.9,96.0,66.0,22.7]               |1.0       |1    |
|[918.7,20.5,92.0,338.0,21.450000000000003]|1.0       |1    |
|[956.6,22.9,95.0,69.0,23.55]              |1.0       |1    |
|[956.3,22.8,93.0,73.0,24.1]               |1.0       |1    |
|[969.0,

Seleciona e exibe as `features`, `prediction` e `Rains` do dataframe resultante das previsões para comparar as previsões com os valores reais.

In [ ]:
# obter os TN, TP, FP, FN
n = df_prediction.count()
tp = df_prediction.filter((F.col('Rains') == 1) & (F.col('prediction') == 1.0)).count()
tn = df_prediction.filter((F.col('Rains') == 0) & (F.col('prediction') == 0.0)).count()
fp = df_prediction.filter((F.col('Rains') == 0) & (F.col('prediction') == 1.0)).count()
fn = df_prediction.filter((F.col('Rains') == 1) & (F.col('prediction') == 0.0)).count()

print("True Positives (TP):", tp)
print("True Negatives (TN):", tn)
print("False Positives (FP):", fp)
print("False Negatives (FN):", fn)
print("Total:", n)

True Positives (TP): 467559
True Negatives (TN): 350826
False Positives (FP): 181671
False Negatives (FN): 66212
Total: 1066268


Este código calcula e exibe True Positives, True Negatives, False Positives, False Negatives, e o total de previsões, indicando o desempenho do modelo ao prever a ocorrência de chuva.

In [15]:
# avaliar o modelo com recurso à AUC 
binary_evaluator = BinaryClassificationEvaluator(labelCol="Rains", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
auc = binary_evaluator.evaluate(df_prediction)
print(f"AUC: {auc}")

# outras métricas de performance do modelo
evaluator = MulticlassClassificationEvaluator(labelCol="Rains", predictionCol="prediction")
accuracy = evaluator.evaluate(df_prediction, {evaluator.metricName: "accuracy"})
recall = evaluator.evaluate(df_prediction, {evaluator.metricName: "weightedRecall"})
precision = evaluator.evaluate(df_prediction, {evaluator.metricName: "weightedPrecision"})
f1 = evaluator.evaluate(df_prediction, {evaluator.metricName: "f1"})

print(f"Accuracy: {accuracy}")
print(f"Recall: {recall}")
print(f"Precision: {precision}")
print(f"F1 Score: {f1}")

AUC: 0.8345782967295283
Accuracy: 0.7675227991461809
Recall: 0.7675227991461808
Precision: 0.780631359569111
F1 Score: 0.7647334317699119


Os resultados das avaliações do modelo de classificação nos dois conjuntos (ao comparar com o notebook anterior) são bastante semelhantes. Ambos apresentam métricas de desempenho como AUC, precisão, recall, e F1 Score com valores muito próximos. Isso indica que o desempenho do modelo é consistente entre os conjuntos de dados de validação utilizados, sugerindo uma performance robusta e estável na previsão da ocorrência de chuva.

In [ ]:
df_prediction.write.mode("overwrite").parquet("classification_model_results.parquet")

Guarda as previsões feitas pelo modelo de classificação no conjunto de dados de validação em um arquivo Parquet chamado "classification_model_results.parquet", substituindo qualquer arquivo existente com o mesmo nome.